In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv('Employee.csv')

In [ ]:
train.isnull().sum()
#there is no null value

In [ ]:
#EDA
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le=LabelEncoder()
train['Attrition']=le.fit_transform(train['Attrition'])

In [ ]:
plt.hist(train['Age'])
plt.show()
import operator
operator.and_(train["Age"]>30,train['Age']<50).sum()

In [ ]:
sns.FacetGrid(train,hue='Attrition',size=5).map(sns.distplot,'Age').add_legend()
operator.and_((train['Age']<=30),(train['Attrition']==0)).sum()  

In [ ]:
plt.hist(train['Attrition'])

In [ ]:
train['BusinessTravel']=le.fit_transform(train['BusinessTravel'])

In [ ]:
#sns.barplot(y=train['Attrition'],x=train['BusinessTravel'])
sns.FacetGrid(train,hue='Attrition',size=5).map(sns.distplot,'BusinessTravel').add_legend()

In [ ]:
train['Department']=le.fit_transform(train['Department'])

In [ ]:
sns.FacetGrid(train,hue='Attrition',size=5).map(plt.scatter,'Department','Age').add_legend()

In [ ]:
sns.FacetGrid(train,hue='Attrition',size=5).map(sns.distplot,'Department').add_legend()

In [ ]:
train['EducationField']=le.fit_transform(train['EducationField'])

In [ ]:
sns.FacetGrid(train,hue='Attrition',size=5).map(sns.distplot,'EducationField').add_legend()

In [ ]:
train['Gender']=le.fit_transform(train['Gender'])
train['JobRole']=le.fit_transform(train['JobRole'])
train['MaritalStatus']=le.fit_transform(train['MaritalStatus'])
train['Over18']=le.fit_transform(train['Over18'])
train['OverTime']=le.fit_transform(train['OverTime'])

In [ ]:
(train['Over18']==0).sum()
#as all values are Y so we can delete this column
train.drop(columns='Over18',inplace=True)

In [ ]:
(train['StandardHours']!=80).sum()
#as there is no other value than 80,so drop this column
train.drop(columns=['StandardHours','EmployeeCount'],inplace=True)

In [ ]:
sns.FacetGrid(train,hue='Attrition',size=5).map(sns.distplot,'JobSatisfaction').add_legend()
#JobSatisfaction 1 'Low' 2 'Medium' 3 'High' 4 'Very High'

In [ ]:
operator.and_(train['Attrition']==1,train["JobSatisfaction"]==1).sum()

In [ ]:
index=train.corrwith(train['Attrition']).sort_values(ascending=False).index
plt.figure(figsize=(15,10))
sns.barplot(x=train.corrwith(train['Attrition']).sort_values(ascending=False),y=index)

In [ ]:
from sklearn.preprocessing import StandardScaler,minmax_scale
y=train.iloc[:,1].values
X=train.drop(columns=['Attrition']).values
X=minmax_scale(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
import xgboost
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import GridSearchCV
accuracy={}

In [ ]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
acc=accuracy_score(y_test,y_pred)
con=confusion_matrix(y_test,y_pred)
accuracy['logisticRegression']=[acc,con]

In [ ]:
sgd=SGDClassifier()
sgd.fit(X_train,y_train)
y_pred=sgd.predict(X_test)
acc=accuracy_score(y_test,y_pred)
con=confusion_matrix(y_test,y_pred)
accuracy['sgdClassifier']=[acc,con]


In [ ]:
xgb=xgboost.XGBClassifier(max_depth=3,n_estimators=100)
xgb.fit(X_train,y_train)
y_pred=xgb.predict(X_test)
acc=accuracy_score(y_test,y_pred)
con=confusion_matrix(y_test,y_pred)
accuracy['xgbclassifier']=[acc,con]

In [ ]:
randomforestclassifier=RandomForestClassifier()
randomforestclassifier.fit(X_train,y_train)
y_pred=randomforestclassifier.predict(X_test)
acc=accuracy_score(y_test,y_pred)
con=confusion_matrix(y_test,y_pred)
accuracy['randomforest']=[acc,con]

In [ ]:
adaboost=AdaBoostClassifier()
adaboost.fit(X_train,y_train)
y_pred=adaboost.predict(X_test)
acc=accuracy_score(y_test,y_pred)
con=confusion_matrix(y_test,y_pred)
accuracy['adaboost']=[acc,con]


In [ ]:
gradientboosting=GradientBoostingClassifier()
gradientboosting.fit(X_train,y_train)
y_pred=gradientboosting.predict(X_test)
acc=accuracy_score(y_test,y_pred)
con=confusion_matrix(y_test,y_pred)
accuracy['gradientboosting']=[acc,con]

In [ ]:
#accuracy of all model
accuracy

In [ ]:
#adaboost hyperparameter tuning
search_grid={'n_estimators':[100,200,300,400,500,600,700,1000,2000],
             'learning_rate':[.001,0.01,.1,.5,1]}
gsearch=GridSearchCV(estimator=AdaBoostClassifier(),param_grid=search_grid,scoring='accuracy',n_jobs=1,cv=10)
gsearch.fit(X_train,y_train)
gsearch.best_params_, gsearch.best_score_
y_pred=gsearch.predict(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
#xgboost hyperparameter tuning
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2),
 'learning_rate':[.001,0.01,.1,1],
 'n_estimators':[100,200,300,400,500,600,700,1000,2000]
}
gsearch1 = GridSearchCV(estimator = xgboost.XGBClassifier( 
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)
gsearch1.best_params_, gsearch1.best_score_
y_pred=gsearch1.predict(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
import statsmodels.api as sm

In [ ]:
x=np.append(arr=np.zeros((X.shape[0],1)).astype(int),values=X,axis=1)
var=np.arange(0,X.shape[1])

In [ ]:
x_opt=x[:,var]

In [ ]:
ols=sm.OLS(endog=y,exog=x_opt).fit()
ols.summary()

In [ ]:
var=np.delete(var,24)
x_opt=x[:,var]
ols=sm.OLS(endog=y,exog=x_opt).fit()
ols.summary()

In [ ]:
var=np.delete(var,25)
x_opt=x[:,var]
ols=sm.OLS(endog=y,exog=x_opt).fit()
ols.summary()

In [ ]:
var=np.delete(var,3)
x_opt=x[:,var]
ols=sm.OLS(endog=y,exog=x_opt).fit()
ols.summary()

In [ ]:
var=np.delete(var,13)
x_opt=x[:,var]
ols=sm.OLS(endog=y,exog=x_opt).fit()
ols.summary()

In [ ]:
var=np.delete(var,19)
x_opt=x[:,var]
ols=sm.OLS(endog=y,exog=x_opt).fit()
ols.summary()

In [ ]:
var=np.delete(var,19)
x_opt=x[:,var]
ols=sm.OLS(endog=y,exog=x_opt).fit()
ols.summary()

In [ ]:
var=np.delete(var,7)
x_opt=x[:,var]
ols=sm.OLS(endog=y,exog=x_opt).fit()
ols.summary()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x_opt,y,test_size=0.2)

In [ ]:
gsearch.best_params_

In [ ]:
gsearch1.best_params_

In [ ]:
xgb=xgboost.XGBClassifier(learning_rate=0.01,
 max_depth= 9,
 min_child_weight= 5,
 n_estimators= 2000)
xgb.fit(X_train,y_train)
y_pred=xgb.predict(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
adaboost=AdaBoostClassifier(learning_rate= 0.1, n_estimators= 700)
adaboost.fit(X_train,y_train)
y_pred=adaboost.predict(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
lr=LogisticRegression(tol=0.01,C=10)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
accuracy_score(y_test,y_pred)